Quiero ver si puedo hacer lo que hice en el notebook data_preprocessing_v0 pero de una forma más general, cosa que yo le pueda dar el dataset y haga todas las operaciones que hice en el otro notebook.

In [0]:
! curl https://meli-data-challenge.s3.amazonaws.com/train.csv.gz --output train.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  659M  100  659M    0     0  70.4M      0  0:00:09  0:00:09 --:--:-- 75.6M


In [0]:
! find . -name '*.csv.gz' -print0 | xargs -0 -n1 gzip -d

In [0]:
! python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 1.7MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=8156dce39c70e7dc74bec953056ab6750f866629e0dddeba16ac5c32a5a2495f
  Stored in directory: /tmp/pip-ephem-wheel-cache-vhm3p3bk/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [0]:
! python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 1.3MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=1e32b88d54e4177a2c1b25f073d33774944e6057d67133dbcab560233c8cefbb
  Stored in directory: /tmp/pip-ephem-wheel-cache-ohvgpcr9/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [0]:
import es_core_news_sm as es_spacy
import pt_core_news_sm as pt_spacy

In [0]:
es_nlp = es_spacy.load()
pt_nlp = pt_spacy.load()

In [0]:
import pandas as pd
import re

In [0]:
def get_languages(data):
  languages = data['language'].unique().tolist()
  return languages

In [0]:
def get_language_splitted_data(data):
  languages = get_languages(data)
  for language in languages:
    if language == 'spanish':
      data_spanish = data[data['language'] == 'spanish']
    if language == 'portuguese':
      data_portuguese = data[data['language'] == 'portuguese']
  return data_spanish, data_portuguese

In [0]:
def choose_nouns(doc):
  nouns = []
  for w in doc:
    if w.pos_ in ['NOUN', 'PROPN']:
      nouns.append(w.text)
  return nouns

In [0]:
def list_to_string(ls):
  str1 = " " 
  return str1.join(ls)

In [0]:
def prepare_data(input_data, language):
  data = input_data
  data['category'] = data['category'].apply(lambda x : '__label__'+x)
  data = data.drop(columns=['label_quality', 'language'])
  
  dim = '(x?)(\s?)(\d+)([.|,]?)((\d+)?)(\s?)(\w*)'
  dim_2 = dim+'(\s?)(x)(\s?)'+dim
  dim_3 = dim_2+'(\s?)(x)(\s?)'+dim
  ev_dim = '|'.join([dim, dim2, dim3])

  data['title'] = data['title'].apply(lambda x : re.sub(ev_dim, '<dimension>', x))
  data['title'] = data['title'].apply(lambda x : x.lower())

  if language == 'spanish':
    data['spacy_doc'] = data['title'].apply(lambda x : es_nlp(x))
  if language == 'portuguese':
    data['spacy_doc'] = data['title'].apply(lambda x : pt_nlp(x))
  
  data['title_nouns'] = data['spacy_doc'].apply(choose_nouns)
  data['title_nouns'] = data['title_nouns'].apply(list_to_string)
  return data

In [0]:
def get_fasttext_input_file(input_data, output_path):
  data = pd.DataFrame()
  data['category'] = input_data['category']
  data['text'] = input_data['title_nouns']
  data.to_csv(output_path, sep=' ', header=False, index=False)
  print('File saved successfully.')  

In [0]:
products = pd.read_csv('train.csv')

In [0]:
small_products = products[0:2000]

In [0]:
small_spanish, small_portuguese = get_language_splitted_data(small_products)

In [79]:
small_spanish_prepared = prepare_data(small_spanish, 'spanish')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [88]:
small_spanish_prepared.head(10)

,title,category,spacy_doc,title_nouns
0,hidrolavadora lavor one bomba aluminio italia,__label__ELECTRIC_PRESSURE_WASHERS,"(hidrolavadora, lavor, one, , bomba, aluminio...",hidrolavadora lavor one bomba aluminio italia
1,placa de sonido - behringer umc,__label__SOUND_CARDS,"(placa, de, sonido, -, behringer, umc)",sonido behringer umc
4,flashes led pestañas luminoso falso pestañas p...,__label__FALSE_EYELASHES,"(flashes, led, pestañas, luminoso, falso, pest...",flashes pestañas partido
9,gatito lunchera neoprene,__label__LUNCHBOXES,"(gatito, lunchera, neoprene)",gatito lunchera
11,rosario contador de billetes uv / mg detecta f...,__label__BILL_COUNTERS,"(rosario, contador, de, billetes, uv, /, mg, d...",rosario contador billetes uv mg
18,battery pack huawei cuerpo de metal usb,__label__PORTABLE_CELLPHONE_CHARGERS,"(battery, pack, huawei, cuerpo, de, metal, usb)",battery pack huawei metal usb
23,portón de chapa mtr marco,__label__GARAGE_DOORS,"(portón, de, chapa, mtr, marco)",portón chapa marco
24,a puertos usb rápido cargador cabeza rápido carg,__label__LABEL_MAKERS,"(a, puertos, usb, rápido, cargador, cabeza, rá...",puertos usb cargador cabeza carg
26,base simil cemento - x,__label__CAKE_TOPPERS,"(base, simil, cemento, -, x)",base simil cemento x
30,maquina para sublimar estampar golde,__label__SCREEN_PRINTERS,"(maquina, para, sublimar, estampar, golde)",maquina golde


In [81]:
output_path = '/content/drive/My Drive/Colab Notebooks/fasttext_input.csv'
get_fasttext_input_file(small_spanish_prepared, output_path)

File saved successfully.


In [0]:
small_verif_set = products[2000:3000]

In [82]:
spa, port = get_language_splitted_data(small_verif_set)
spa_prep = prepare_data(spa, 'spanish')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
spa_prep.head(10)

,title,category,spacy_doc,title_nouns
2000,gub gc bmx ultraligero pedales montaña,__label____label__BICYCLE_PEDALS,"(gub, gc, bmx, ultraligero, pedales, montaña)",gub gc bmx ultraligero montaña
2002,palos para bombo criollo nuciforo,__label____label__BOMBOS,"(palos, para, bombo, criollo, nuciforo)",palos bombo criollo nuciforo
2006,geib gator profesional buttercut cizalla recta...,__label____label__ANIMAL_CLIPPERS,"(geib, gator, profesional, buttercut, cizalla,...",geib gator profesional buttercut cizalla
2009,camara olympus am-,__label____label__ANALOG_CAMERAS,"(camara, olympus, am-)",olympus am-
2014,joe rocket --- guantes - tamaño : - guante so,__label____label__MOTORCYCLE_GLOVES,"(joe, rocket, ---, guantes, -, tamaño, :, -, g...",joe rocket guantes tamaño
2016,multifuncional sistema de conducción de seguri...,__label____label__VEHICLE_TRACKERS,"(multifuncional, sistema, de, conducción, de, ...",multifuncional sistema conducción seguridad coche
2021,servilletero mu,__label____label__NAPKIN_HOLDERS,"(servilletero, mu)",mu
2022,carro tipo zorra,__label____label__HAND_TRUCKS,"(carro, tipo, zorra)",tipo zorra
2023,encordado guitarra clasica criolla martin blust,__label____label__GUITAR_STRINGS,"(encordado, guitarra, clasica, criolla, , mar...",guitarra martin blust
2024,soldadora inverter + mascara fotosensible,__label____label__SOLDERING_MACHINES,"(soldadora, inverter, +, mascara, fotosensible)",inverter +


Por algunos títulos que vemos acá, vemos que hay más cosas que limpiar con respecto a los títulos:
*   signos de puntuación junto a palabras (a veces unen palabras)
*   unidades de medida (hay desde m, mm, etc hasta uv y mg)
*   palabras como: vendo, oferta, cuotas, kit, combo
*   errores de ortografía, tipeo

Aparte, hay cosas que spaCy flasheó y no contó como sustantivos. Por ejemplo servilletero, carro, pescadora, entre otros.